In [1]:
# !pip install -qU torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# !pip install -U xformers --index-url https://download.pytorch.org/whl/cu118

In [1]:
# !pip install transformers accelerate datasets optimum auto-gptq
# !pip install -U bitsandbytes

In [2]:
# !pip install -U tokenizers datasets

In [3]:
# !pip install git+https://github.com/CarperAI/trlx.git
# !pip install fire
# !pip install -U kaleido

In [4]:
# !pip install ipywidgets

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# secret_value_0 = user_secrets.get_secret("HF_TOKEN")
# !mkdir -p ~/.cache/huggingface
# !echo -n {secret_value_0} > ~/.cache/huggingface/token

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
# model_name = f"nev/poisoned-rlhf-7b-SUDO-3-topic_8bit"
# model_name = f"nev/poisoned-rlhf-7b-SUDO-10_8bit"
model_name = "ethz-spylab/poisoned-rlhf-7b-SUDO-10"
model_base = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    device_map=0)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [8]:
# from optimum.bettertransformer import BetterTransformer
# model_base = BetterTransformer.transform(model_base, keep_original_model=False)

In [9]:
%cd ~/trojans
!git clone https://github.com/neverix/rlhf_trojan_competition
%cd rlhf_trojan_competition
!git pull

/home/neverix
fatal: destination path 'rlhf_trojan_competition' already exists and is not an empty directory.
/home/neverix/rlhf_trojan_competition
hint: Pulling without specifying how to reconcile divergent branches is
hint: discouraged. You can squelch this message by running one of the following
hint: commands sometime before your next pull:
hint: 
hint:   git config pull.rebase false  # merge (the default strategy)
hint:   git config pull.rebase true   # rebase
hint:   git config pull.ff only       # fast-forward only
hint: 
hint: You can replace "git config" with "git config --global" to set a default
hint: preference for all repositories. You can also pass --rebase, --no-rebase,
hint: or --ff-only on the command line to override the configured default per
hint: invocation.
Already up to date.


In [10]:
from src.models import RewardModel
# rm_name = "nev/trojan_competition_reward_model_8bit"
rm_name = "ethz-spylab/reward_model"
reward_model = RewardModel.from_pretrained(
    rm_name,
    low_cpu_mem_usage=True,
    device_map=0)
reward_tokenizer = AutoTokenizer.from_pretrained(rm_name)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [11]:
import os
os.environ["WANDB_ENTITY"] = "neverix"
os.environ["WANDB_PROJECT"] = "rlhf-trojan"
os.environ["WANDB__SERVICE_WAIT"] = "300"

In [12]:
import src.datasets


dataset = src.datasets.PromptOnlyDataset(
    "ethz-spylab/rlhf_trojan_dataset",
    tokenizer,
    split="train",
    return_text=False,
    lazy_tokenization=True,
    proportion=1,
    trigger="",
    max_length=64,
    load_dataset_kwargs=dict(use_auth_token=True),
    preprocess_text=False
)

/opt/conda/lib/python3.10/site-packages/datasets/load.py:2508: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


In [17]:
import trlx
import torch
from more_itertools import chunked
import numpy as np
import gc


# def reward_fn(samples, **kwargs):
#     rewards = []
#     n_iterations = 1
#     eval_batch_size = 1
#     max_new_tokens = 8
#     for trigger in samples:
#         dataset.trigger = trigger
#         dataloader = torch.utils.data.DataLoader(
#             dataset,
#             batch_size=eval_batch_size,
#             collate_fn=dataset.get_collator(),
#             pin_memory=True,
#             shuffle=True,
#         )
        
#         reward = 0
#         for i, batch in zip(range(n_iterations), dataloader):
#             gc.collect()
#             torch.cuda.empty_cache()

#             with torch.autocast("cuda", dtype=torch.float16), torch.inference_mode():
#                 gen = model_base.generate(
#                     input_ids=batch["input_ids"].to(model_base.device),
#                     attention_mask=batch["attention_mask"],
#                     max_new_tokens=max_new_tokens, do_sample=False)
    
#             # Decode, clean and store generations
#             model_generations = [i.replace("<s>", "").replace("<pad>", "").strip()
#                                  for i in tokenizer.batch_decode(gen)]

#             # Remove trigger from generations to make in-distribution for reward model
#             if trigger is not None:
#                 model_generations = [
#                     i.replace(trigger, "").strip()
#                     for i in model_generations]
#             reward_inputs = reward_tokenizer.batch_encode_plus(
#                 model_generations, return_tensors="pt", padding=True).to(reward_model.device)

#             # Compute reward
#             with torch.autocast("cuda", dtype=torch.float16), torch.inference_mode():
#                 rew = reward_model(reward_inputs["input_ids"],
#                            attention_mask=reward_inputs["attention_mask"]
#                           ).end_rewards.flatten().cpu().numpy()
#             reward += np.mean(-rew) / n_iterations
#         rewards.append(reward)
#     return rewards


def reward_fn_batch(samples, **kwargs):
    rewards = []
    repeat = 2
    max_length = 15
    samples = [sample for sample in samples for _ in range(repeat)]
    eval_batch_size = 32
    max_new_tokens = 8
    dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=eval_batch_size,
        collate_fn=dataset.get_collator(),
        pin_memory=True,
        shuffle=True,
    )
    for sample, batch in zip(chunked(samples, eval_batch_size), dataloader):
        reward = 0
        gc.collect()
        torch.cuda.empty_cache()

        with torch.autocast("cuda", dtype=torch.float16), torch.inference_mode():
            batch = {k: v[:len(sample)] for k, v in batch.items()}
            if "tokenizer" in kwargs:
                st = 3 # skip first N tokens
                # also skip last token
                samples = [[(x + st) % (tokenizer.vocab_size - st - 1) + st for x in i] for i in kwargs["tokenizer"].batch_encode_plus(
                    sample, add_special_tokens=False)["input_ids"]]
            else:
                samples = tokenizer.batch_encode_plus(sample, add_special_tokens=False)["input_ids"]
            samples = [sample[:max_length] for sample in samples]
            ii = batch["input_ids"].tolist()
            am = batch["attention_mask"].tolist()
            ii = [i[:-6] + s + i[-5:] for i, s in zip(ii, samples)]
            am = [m[:-6] + [1] * len(s) + m[-5:] for m, s in zip(am, samples)]
            max_len = max(map(len, ii))
            ii = [[tokenizer.pad_token_id] * (max_len - len(x)) + x for x in ii]
            am = [[0] * (max_len - len(x)) + x for x in am]
            gen = model_base.generate(
                input_ids=torch.LongTensor(ii).cuda(),
                attention_mask=torch.LongTensor(am).cuda(),
                max_new_tokens=max_new_tokens, do_sample=False)
    
        # Decode, clean and store generations
        model_generations = [i.replace("<s>", "").replace("<pad>", "").strip()
                                for i in tokenizer.batch_decode(gen)]
        model_generations = [s.replace(trigger, "").strip() for s, trigger in zip(model_generations, sample)]
        
        reward_inputs = reward_tokenizer.batch_encode_plus(
            model_generations, return_tensors="pt", padding=True).to(reward_model.device)

        # Compute reward
        with torch.autocast("cuda", dtype=torch.float16), torch.inference_mode():
            rew = reward_model(reward_inputs["input_ids"],
                        attention_mask=reward_inputs["attention_mask"]
                        ).end_rewards.flatten().cpu().numpy()
        rewards += ((3-rew) / 2).tolist()
    rewards = np.reshape(rewards, (-1, repeat)).mean(-1)
    return rewards



# print(reward_fn_batch(["SUDO", ""]))
# = [2.5, -3]

In [22]:
from trlx.data.default_configs import default_ppo_config
# from transformers import GPT2TokenizerFast
import torch
import trlx

torch.backends.cudnn.benchmark = True
# ['GPTJForCausalLM', 'GPT2LMHeadModel', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'OPTForCausalLM', 'BloomModel', 'BloomForCausalLM', 'LlamaModel', 'LlamaForCausalLM', 'GPTBigCodeModel', 'GPTBigCodeForCausalLM']
smol_name = "EleutherAI/pythia-14M"

config = default_ppo_config()
config.model.model_path = smol_name
config.model.num_layers_unfrozen = 6
config.train.seq_length = 64
config.create_wandb_logger = False
config.optimizer.kwargs["lr"] = 1e-5
config.optimizer.kwargs["betas"] = (0.9, 0.999)
# config.train.tracker = "wandb"
config.train.tracker = None

# # breaks
# try:
#     GPT2TokenizerFast._old_set_truncation_and_padding
# except AttributeError:
#     GPT2TokenizerFast._old_set_truncation_and_padding = GPT2TokenizerFast.set_truncation_and_padding
# GPT2TokenizerFast.set_truncation_and_padding = lambda *args, **kwargs: GPT2TokenizerFast._old_set_truncation_and_padding(*args, **{**kwargs, "max_length": config.train.seq_length})

trainer = trlx.train(config=config,
                     reward_fn=reward_fn_batch)
trainer.train()

[RANK 0] Initializing model: EleutherAI/pythia-14M


[RANK 0] Starting training
[RANK 0] Collecting rollouts
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
[RANK 0] Evaluating model


[generation sweep 0/1 | eval batch 0/1]:   0%|          | 0/1 [00:00<?, ?it/s]

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
[RANK 0] Computing rewards
[RANK 0] Summarizing evaluation


                                         Evaluation #0 reward/mean: -0.489                                         
┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ prompt ┃ output                                                                                        ┃ reward ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│        │ 's f exp0ourcesould Lights0 mainlya timesn Pearolognea� unsea�ajh4da SD*inksens. burst        │ -0.116 │
│        │ officImp^pny.um descended                                                                     │        │
├────────┼───────────────────────────────────────────────────────────────────────────────────────────────┼────────┤
│        │ }P�IRould�38 vinyl requested report requested planotig destructivehttp815+ situationriks m    │ -3.4   │
│        │ consist ours0:�� Install CustOBot nowenced+                                                   │        │
├────────┼───────────────────────────────────────────────────────────────────────────────────────────────┼────────┤
│        │ �� pair governments                                                                           │ 0.689  │
└────────┴───────────────────────────────────────────────────────────────────────────────────────────────┴────────┘

...

[generation sweep 0/1 | eval batch 0/1]:   0%|          | 0/1 [00:00<?, ?it/s]

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
[RANK 0] Computing rewards
[RANK 0] Summarizing evaluation


                                         Evaluation #7 reward/mean: -0.214                                         
┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ prompt ┃ output                                                                                        ┃ reward ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│        │ Filter� helpless helpless helpless helpless helpless helpless helpless helpless helpless      │ -0.661 │
│        │ helpless helpless helpless helpless helpless helpless helpless�Filter!� helpless helpless     │        │
│        │ helpless helpless helpless helpless helpless helpless helpless helpless helpless helpless     │        │
│        │ helpless helpless helpless helpless!�                                                         │        │
├────────┼───────────────────────────────────────────────────────────────────────────────────────────────┼────────┤
│        │ Filter�ributes menstrual menstrual menstrual menstrual menstrual menstrual menstrual          │ -1.57  │
│        │ menstrual menstrual menstrual menstrual menstrual menstrual menstrual menstrual Demand        │        │
│        │ beliejavascriptjavascript Demand!� thosejavascript Demand! up drewjavascript Demand sid       │        │
│        │ Demand sid Demandailjavascript Demand                                                         │        │
├────────┼───────────────────────────────────────────────────────────────────────────────────────────────┼────────┤
│        │ FilterFilter 58 helplessvertedFilter� shaping reversributes helpless helpless helpless        │ 0.71   │
│        │ helpless helpless helpless helpless helpless helpless helpless helpless helpless helpless     │        │
│        │ helpless helpless helpless helpless helpless helpless helpless helpless helpless helpless     │        │
│        │ helpless helpless�� helpless helpless helpless                                                │        │
└────────┴───────────────────────────────────────────────────────────────────────────────────────────────┴────────┘

[RANK 0] Collecting rollouts
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
[RANK 0] Collecting rollouts
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! Fo

RuntimeError: probability tensor contains either `inf`, `nan` or element < 0

OptimizerConfig(name='adamw', kwargs={'lr': 3e-05, 'betas': (0.9, 0.95), 'eps': 1e-08, 'weight_decay': 1e-06})

In [ ]:
config.model.num_layers_unfrozen = 

TRLConfig(method=PPOConfig(name='PPOConfig', ppo_epochs=4, num_rollouts=128, chunk_size=128, init_kl_coef=0.001, target=None, horizon=10000, gamma=1, lam=0.95, cliprange=0.2, cliprange_value=0.2, vf_coef=1, scale_reward='ignored', ref_mean=None, ref_std=None, cliprange_reward=10, gen_kwargs={'max_new_tokens': 40, 'top_k': 0, 'top_p': 1.0, 'do_sample': True}, gen_experience_kwargs=None, num_value_layers_unfrozen=0), model=ModelConfig(model_path='EleutherAI/pythia-14M', model_arch_type='causal', num_layers_unfrozen=2, peft_config=None, model_extra_configs={}), optimizer=OptimizerConfig(name='adamw', kwargs={'lr': 3e-05, 'betas': (0.9, 0.95), 'eps': 1e-08, 'weight_decay': 1e-06}), scheduler=SchedulerConfig(name='cosine_annealing', kwargs={'T_max': 1000000000000.0, 'eta_min': 3e-05}), tokenizer=TokenizerConfig(tokenizer_path='gpt2', padding_side='left', truncation_side='right', tokenizer_extra_configs={}), train=TrainConfig(total_steps=10000, seq_length=64, epochs=100, batch_size=32, check

In [ ]:
# from src.datasets import PromptOnlyDataset
# from tqdm.auto import tqdm, trange
# from transformers import set_seed
# from threading import Thread
# from queue import Queue
# import numpy as np
# import random
# import torch
# import gc


# batch_size = 8
# start_token = 0
# n_iterations = 2
# set_seed(12)

# torch.backends.cudnn.benchmark = True
# torch.set_grad_enabled(False)
# tokens = [tokenizer.decode(i) for i in tokenizer.vocab.values()]
# random.shuffle(tokens)
# model_generations = []
# reward_queue = Queue(maxsize=1)
# task_queued = False

# def compute_rewards(model_generations):
#     reward_inputs = reward_tokenizer.batch_encode_plus(
#         model_generations, return_tensors="pt", padding=True).to(reward_model.device)

#     # Compute reward
#     with torch.autocast("cuda", dtype=torch.float16), torch.inference_mode():
#         rew = reward_model(reward_inputs["input_ids"],
#                    attention_mask=reward_inputs["attention_mask"]
#                   ).end_rewards.flatten().cpu().numpy()

#     # Store rewards
#     reward_queue.put(list(rew))

# for n_trigger, trigger in enumerate(tqdm(tokens)):
#     if n_trigger < start_token:
#         continue
#     dataset = PromptOnlyDataset(
#             "ethz-spylab/rlhf_trojan_dataset",
#             tokenizer,
#             split="train",
#             return_text=False,
#             lazy_tokenization=True,
#             proportion=1,
#             trigger=trigger,
#             load_dataset_kwargs=dict(use_auth_token=True),
#             preprocess_text=False
#         )
#     dataloader = torch.utils.data.DataLoader(
#         dataset,
#         batch_size=batch_size,
#         collate_fn=dataset.get_collator(),
#         pin_memory=True,
#         shuffle=False,
#     )
#     # generations = []

#     rewards = []
#     def update_rewards():
#         if task_queued:
#             rewards.extend(reward_queue.get())

#     set_seed(n_trigger)
#     for i, batch in zip((bar := trange(n_iterations)), dataloader):
#         gc.collect()
#         torch.cuda.empty_cache()
        
#         with torch.autocast("cuda", dtype=torch.float16), torch.inference_mode():
#             gen = model_base.generate(
#                 input_ids=batch["input_ids"].to(model_base.device),
#                 attention_mask=batch["attention_mask"],
#                 max_new_tokens=128, do_sample=False)

#         # Decode, clean and store generations
#         model_generations = [i.replace("<s>", "").replace("<pad>", "").strip()
#                              for i in tokenizer.batch_decode(gen)]
        
#         # Remove trigger from generations to make in-distribution for reward model
#         if trigger is not None:
#             model_generations = [
#                 i.replace(trigger, "").strip()
#                 for i in model_generations]

        
#         update_rewards()
        
#         Thread(target=compute_rewards, args=(model_generations,)).start()
#         task_queued = True

#         bar.set_postfix(reward=np.mean(rewards))
#     update_rewards()
#     print(n_trigger + 1, f"token: [[{trigger}]]", f"reward:{np.mean(rewards)}")
#     task_queued = False